In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 1) Load data (ajuste les noms si besoin via l’onglet Data)
train = pd.read_csv("/kaggle/input/predict-closed-questions-on-stack-overflow/train.csv")
test  = pd.read_csv("/kaggle/input/predict-closed-questions-on-stack-overflow/public_leaderboard.csv")

# 2) Features simples: titre + corps
Xtr_text = (train["Title"].fillna("") + " " + train["BodyMarkdown"].fillna(""))
Xte_text = (test["Title"].fillna("")  + " " + test["BodyMarkdown"].fillna(""))

tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
Xtr = tfidf.fit_transform(Xtr_text)
Xte = tfidf.transform(Xte_text)

# 3) Modèle
clf = LogisticRegression(max_iter=2000, multi_class="multinomial", solver="saga")
clf.fit(Xtr, train["OpenStatus"])

# 4) Probas → submission.csv (ordre EXACT des classes)
classes = ["not a real question","not constructive","off topic","open","too localized"]
proba = clf.predict_proba(Xte)
sub = pd.DataFrame(proba, columns=clf.classes_).reindex(columns=classes, fill_value=0)

# inclure 'id' si présent, sinon Kaggle l’ignorera (facultatif)
if "id" in test.columns:
    sub.insert(0, "id", test["id"].values)

sub.to_csv("submission.csv", index=False)
print("submission.csv écrit.")
